In [31]:
# load the sonar dataset
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from autokeras import StructuredDataClassifier

# load dataset
#url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv'
dataframe = pd.read_csv("unsw-small.csv")

In [32]:
dataframe

,Unnamed: 0,duration,orig_pkts,resp_pkts,orig_ip_bytes,resp_ip_bytes,rate,attack_cat,label
0,0,0.121478,6,4,258,172,74.087490,Normal,0
1,1,0.649902,14,38,734,42014,78.473372,Normal,0
2,2,1.623129,8,16,364,13186,14.170161,Normal,0
3,3,1.681642,12,12,628,770,13.677108,Normal,0
4,4,0.449454,10,6,534,268,33.373826,Normal,0
...,...,...,...,...,...,...,...,...,...
257668,257668,0.000005,2,0,104,0,200000.005100,Normal,0
257669,257669,1.106101,20,8,18062,354,24.410067,Normal,0
257670,257670,0.000000,1,0,46,0,0.000000,Normal,0
257671,257671,0.000000,1,0,46,0,0.000000,Normal,0


In [33]:
del dataframe['Unnamed: 0']
del dataframe['rate']
del dataframe['attack_cat']

In [34]:
print(dataframe.shape)
# split into input and output elements
data = dataframe.values
X, y = data[:, :-1], data[:, -1]

print(X.shape, y.shape)


(257673, 6)
(257673, 5) (257673,)


In [35]:
X

array([[1.214780e-01, 6.000000e+00, 4.000000e+00, 2.580000e+02,
        1.720000e+02],
       [6.499020e-01, 1.400000e+01, 3.800000e+01, 7.340000e+02,
        4.201400e+04],
       [1.623129e+00, 8.000000e+00, 1.600000e+01, 3.640000e+02,
        1.318600e+04],
       ...,
       [0.000000e+00, 1.000000e+00, 0.000000e+00, 4.600000e+01,
        0.000000e+00],
       [0.000000e+00, 1.000000e+00, 0.000000e+00, 4.600000e+01,
        0.000000e+00],
       [9.000000e-06, 2.000000e+00, 0.000000e+00, 1.040000e+02,
        0.000000e+00]])

In [36]:
# basic data preparation
X = X.astype('float32')
y = LabelEncoder().fit_transform(y)
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(172640, 5) (85033, 5) (172640,) (85033,)


In [37]:
# define the search
search = StructuredDataClassifier(max_trials=25)

INFO:tensorflow:Reloading Oracle from existing project ./structured_data_classifier/oracle.json
INFO:tensorflow:Reloading Tuner from ./structured_data_classifier/tuner0.json


In [38]:
# perform the search
search.fit(x=X_train, y=y_train, verbose=1)

Trial 23 Complete [00h 01m 24s]
val_accuracy: 0.8791995048522949

Best val_accuracy So Far: 0.8836596608161926
Total elapsed time: 00h 45m 58s
INFO:tensorflow:Oracle triggered exit
Epoch 1/6
5395/5395 [==============================] - 7s 1ms/step - loss: 0.2979 - accuracy: 0.8734
Epoch 2/6
5395/5395 [==============================] - 7s 1ms/step - loss: 0.2368 - accuracy: 0.8896
Epoch 3/6
5395/5395 [==============================] - 6s 1ms/step - loss: 0.2179 - accuracy: 0.8964
Epoch 4/6
5395/5395 [==============================] - 6s 1ms/step - loss: 0.2094 - accuracy: 0.8992
Epoch 5/6
5395/5395 [==============================] - 6s 1ms/step - loss: 0.2050 - accuracy: 0.8999
Epoch 6/6
5395/5395 [==============================] - 7s 1ms/step - loss: 0.2006 - accuracy: 0.9016
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are ap

In [39]:
# evaluate the model
loss, acc = search.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.3f' % acc)

Accuracy: 0.876


In [ ]:
# use the model to make a prediction
row = [0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032]
X_new = asarray([row]).astype('float32')
yhat = search.predict(X_new)
print('Predicted: %.3f' % yhat[0])

In [40]:
# get the best performing model
model = search.export_model()

In [41]:
# summarize the loaded model
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 5)                 0         
_________________________________________________________________
normalization (Normalization (None, 5)                 11        
_________________________________________________________________
dense (Dense)                (None, 1024)              6144      
_________________________________________________________________
re_lu (ReLU)                 (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 256)              

In [52]:
# save the best performing model to file
model.save('model_AI4Sec', overwrite=True, include_optimizer=True)

INFO:tensorflow:Assets written to: model_AI4Sec/assets


In [59]:
import tensorflow as tf
import keras
import numpy as np

In [61]:
loaded = keras.models.load_model("model_AI4Sec")

In [57]:
loaded.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 5)                 0         
_________________________________________________________________
normalization (Normalization (None, 5)                 11        
_________________________________________________________________
dense (Dense)                (None, 1024)              6144      
_________________________________________________________________
re_lu (ReLU)                 (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
re_lu_1 (ReLU)               (None, 256)              

NameError: name 'input_arr' is not defined